In [18]:
from __future__ import division, print_function, absolute_import

import tflearn
from tflearn.data_utils import shuffle, to_categorical
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.estimator import regression
from tflearn.data_preprocessing import ImagePreprocessing
from tflearn.data_augmentation import ImageAugmentation

import tensorflow as tf
tf.reset_default_graph()

print("Data loading and preprocessing")
from tflearn.datasets import cifar10
(X, Y), (X_test, Y_test) = cifar10.load_data()
X, Y = shuffle(X, Y)
Y = to_categorical(Y, 10)
Y_test = to_categorical(Y_test, 10)

print("Real-time data preprocessing")
img_prep = ImagePreprocessing()
img_prep.add_featurewise_zero_center()
img_prep.add_featurewise_stdnorm()

print("Real-time data augmentation")
img_aug = ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_rotation(max_angle=25.)

print("Convolutional network building")
network = input_data(shape=[None, 32, 32, 3],
                     data_preprocessing=img_prep,
                     data_augmentation=img_aug)
network = conv_2d(network, 32, 3, activation='relu')
network = max_pool_2d(network, 2)
network = conv_2d(network, 64, 3, activation='relu')
network = conv_2d(network, 64, 3, activation='relu')
network = max_pool_2d(network, 2)
network = fully_connected(network, 512, activation='relu')
network = dropout(network, 0.5)
network = fully_connected(network, 10, activation='softmax')
network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)

print("something else")
model = tflearn.DNN(network, tensorboard_verbose=0)
print("Actually train using classifier")
model.fit(X, Y, n_epoch=50, shuffle=True, validation_set=(X_test, Y_test),
          show_metric=True, batch_size=96, run_id='cifar10_cnn')

Training Step: 26049  | total loss: 0.35344 | time: 117.346s
| Adam | epoch: 050 | loss: 0.35344 - acc: 0.8746 -- iter: 49920/50000
Training Step: 26050  | total loss: 0.35158 | time: 125.284s
| Adam | epoch: 050 | loss: 0.35158 - acc: 0.8757 | val_loss: 0.65917 - val_acc: 0.8163 -- iter: 50000/50000
--


In [20]:
model.save("classifier")

INFO:tensorflow:/Users/jacenq/Git/Tensorflow/classifier2 is not in all_model_checkpoint_paths. Manually adding it.


In [22]:
model.load("classifier.tfl")

INFO:tensorflow:Restoring parameters from /Users/jacenq/Git/Tensorflow/classifier.tfl


In [58]:
import scipy
import numpy as np

img = scipy.ndimage.imread("marcus.png", mode="RGB")
img = scipy.misc.imresize(img, (32, 32), interp="bicubic").astype(np.float32, casting='unsafe')
scipy.misc.imsave("marcus_resized.jpg", img);

In [59]:
prediction = model.predict([img])

In [60]:
print(prediction)

[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]]
